# YAGO 3-10+

In [2]:
from glob import glob
from tqdm import tqdm
from pickle import dump, load, HIGHEST_PROTOCOL
from os import makedirs
from os.path import exists
from urllib.request import urlopen
from bz2 import open as bz2open
from shutil import copy as copy_file
import gzip
import tarfile
from subprocess import run, PIPE

from random import randint

!pip install py7zr

In [3]:
draft_folder=f"../draft/YAGO3-10+"
destination_folder=f"."

In [4]:
draft_folder=f"{draft_folder}{'' if draft_folder.endswith('/') else '/'}"
destination_folder=f"{destination_folder}{'' if destination_folder.endswith('/') else '/'}"

YAGO3_10_LP_DATASET_FOLDER="https://github.com/TimDettmers/ConvE/raw/refs/heads/master/"
YAGO3_10_LP_DATASET_FILE="YAGO3-10.tar.gz"
YAGO3_10_ARCHIVE_URL_FOLDER = "https://yago-knowledge.org/data/yago3/"
YAGO3_10_ARCHIVE_URL_FILE="yago-3.0.2-turtle-simple.7z"

YAGO3_10_LP=f"{draft_folder}LP/"
YAGO3_10=f"{draft_folder}YAGO3-10/"
YAGO3=f"{draft_folder}YAGO3/"

dest_txt_folder=destination_folder
dest_pkl_folder=f"{destination_folder}pickle/"

In [5]:
for folder in [
    draft_folder,
    dest_txt_folder,
    dest_pkl_folder,
    YAGO3_10_LP,
    YAGO3_10,
    YAGO3
]: 
    makedirs(folder, exist_ok=True)

In [6]:
if not exists(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE[:-7]}/train.txt") \
    or not exists(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE[:-7]}/test.txt") \
    or not exists(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE[:-7]}/train.txt"):

    if not exists(f"{YAGO3_10_LP_DATASET_FOLDER}{YAGO3_10_LP_DATASET_FILE}"):

        with urlopen(f"{YAGO3_10_LP_DATASET_FOLDER}{YAGO3_10_LP_DATASET_FILE}") as file_online:
            with open(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE}", "wb") as file_local:
                file_local.write(file_online.read())
    
    tar = tarfile.open(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE}", 'r:gz')
    tar.extractall(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE[:-7]}")
    tar.close()

C:\Users\nrobert\AppData\Local\Temp\ipykernel_2864\3085004144.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(f"{draft_folder}{YAGO3_10_LP_DATASET_FILE[:-7]}")


In [7]:
if not exists(f"{YAGO3}{YAGO3_10_ARCHIVE_URL_FILE}"):
    with urlopen(f"{YAGO3_10_ARCHIVE_URL_FOLDER}{YAGO3_10_ARCHIVE_URL_FILE}") as file_online:
        with open(f"{YAGO3}{YAGO3_10_ARCHIVE_URL_FILE}", "wb") as file_local:
            file_local.write(file_online.read())

In [8]:
if not exists(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTypes.ttl") \
    or not exists(f"{YAGO3}yago-3.0.2-turtle-simple/yagoSchema.ttl") \
    or not exists(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTaxonomy.ttl"):
    makedirs(f"{YAGO3}{YAGO3_10_ARCHIVE_URL_FILE}"[:-3], exist_ok=True)
    
    packed_file=f"{YAGO3}{YAGO3_10_ARCHIVE_URL_FILE}"
    unpack_folder=packed_file[:-3]
    
    print(f"py7zr x {packed_file} {unpack_folder}")
    unpack_command = run(["py7zr", "x", packed_file, unpack_folder], stdout = PIPE, stderr = PIPE)
    
    print("Standard output:")
    print(unpack_command.stdout)
    
    if "stderr" in dir(unpack_command):
        print("Standard error:")
        print(unpack_command.stderr)

py7zr x ../draft/YAGO3-10+/YAGO3/yago-3.0.2-turtle-simple.7z ../draft/YAGO3-10+/YAGO3/yago-3.0.2-turtle-simple
Standard output:
b''
Standard error:
b''


# ent2id, rel2id, class2id

In [10]:
rels = set([])
ents = set([])

for file in [
    "train.txt",
    "test.txt",
    "valid.txt"
]:
    file_path=f"{YAGO3_10}{file}"
    nb_lines=sum(1 for _ in open(file_path, "r", encoding="utf-8"))
    with open(file_path, "r", encoding="utf-8") as split:
        with tqdm(enumerate(split), total=nb_lines) as bar:
            bar.set_description(f"Extracting entities and relations from {file}")
            for i, line in bar:
                s, p, o = line.strip().split("\t")
    
                s=f"http://yago-knowledge.org/resource/{s}"
                p=f"http://yago-knowledge.org/resource/{p}"
                o=f"http://yago-knowledge.org/resource/{o}"
    
                ents.add(s)
                ents.add(o)
                rels.add(p)

Extracting entities and relations from valid.txt: 100%|████████████████████████| 5000/5000 [00:00<00:00, 321994.78it/s]


In [11]:
len(list(ents)), len(list(rels))

(123182, 37)

In [12]:
rel2id = {rel: i for i, rel in enumerate(list(rels))}
id2rel = {i: rel for rel, i in rel2id.items()}

with open(f"{dest_pkl_folder}rel2id.pkl", "wb") as handle:
    dump(rel2id, handle)

ent2id = {ent: i for i, ent in enumerate(list(ents))}
id2ent = {i: ent for ent, i in ent2id.items()}

with open(f"{dest_pkl_folder}ent2id.pkl", "wb") as handle:
    dump(ent2id, handle)

In [13]:
{k: v for k, v in list(ent2id.items())[:10]}

{'http://yago-knowledge.org/resource/Aleksei_Bugayev': 0,
 'http://yago-knowledge.org/resource/Vincent_Planté': 1,
 'http://yago-knowledge.org/resource/Anton_Weissenbacher': 2,
 'http://yago-knowledge.org/resource/Mindelo': 3,
 'http://yago-knowledge.org/resource/Lonoke,_Arkansas': 4,
 'http://yago-knowledge.org/resource/Aizuwakamatsu,_Fukushima': 5,
 'http://yago-knowledge.org/resource/José_Fábio_da_Silva': 6,
 'http://yago-knowledge.org/resource/Borough_of_Newcastle-under-Lyme': 7,
 'http://yago-knowledge.org/resource/Mazovia': 8,
 'http://yago-knowledge.org/resource/Andapa_District': 9}

In [14]:
{k: v for k, v in list(rel2id.items())[:10]}

{'http://yago-knowledge.org/resource/livesIn': 0,
 'http://yago-knowledge.org/resource/hasMusicalRole': 1,
 'http://yago-knowledge.org/resource/participatedIn': 2,
 'http://yago-knowledge.org/resource/wroteMusicFor': 3,
 'http://yago-knowledge.org/resource/worksAt': 4,
 'http://yago-knowledge.org/resource/created': 5,
 'http://yago-knowledge.org/resource/playsFor': 6,
 'http://yago-knowledge.org/resource/influences': 7,
 'http://yago-knowledge.org/resource/hasGender': 8,
 'http://yago-knowledge.org/resource/isLocatedIn': 9}

### Encode train, test, valid splits

In [16]:
inverse_predicate_offset = max(rel2id.values()) + 1
for split in ["train", "test", "valid"]:
    nb_lines=sum(1 for _ in open(f"{YAGO3_10}{split}.txt", "r", encoding="utf-8"))
    with open(f"{YAGO3_10}{split}.txt", "r", encoding='utf-8') as r:
        with open(f"{dest_txt_folder}{split}2id.txt", "w+", encoding='utf-8') as w:
            with tqdm(enumerate(r), total=nb_lines) as bar:
                bar.set_description(f"Encoding {split}2id.txt")
                for i, line in bar:
                    s, p, o = line.strip().split("\t")

                    s=f"http://yago-knowledge.org/resource/{s}"
                    p=f"http://yago-knowledge.org/resource/{p}"
                    o=f"http://yago-knowledge.org/resource/{o}"
                    
                    encoded_s = ent2id[s]
                    encoded_p = rel2id[p]
                    encoded_o = ent2id[o]
                    
                    w.write(f"{encoded_s}\t{encoded_p}\t{encoded_o}\n")   
                    
        copy_file(f"{dest_txt_folder}{split}2id.txt", f"{dest_txt_folder}{split}2id_inv.txt")
        with open(f"{dest_txt_folder}{split}2id.txt", "r", encoding='utf-8') as r:
            with open(f"{dest_txt_folder}{split}2id_inv.txt", "a", encoding='utf-8') as a:
                with tqdm(enumerate(r), total=nb_lines) as bar:
                    bar.set_description(f"Encoding {split}2id_inv.txt")
                    for i, line in bar:
                        s, p, o = line.strip().split("\t")
                        p = int(p)
                        a.write(f"{o}\t{p+inverse_predicate_offset}\t{s}\n")

Encoding valid2id_inv.txt: 100%|███████████████████████████████████████████████| 5000/5000 [00:00<00:00, 321910.76it/s]


# observed_heads_original_kg, observed_tails_original_kg, observed_heads_inv, observed_tails_inv

In [18]:
observed_heads_original_kg={}
observed_heads_inv={}
observed_tails_original_kg={}
observed_tails_inv={}

def observe(d, a, b, c):
    if not a in d.keys():
        d[a]={}

    if not b in d[a].keys():
        d[a][b]=[c]
    else:
        d[a][b].append(c)


for split in ["train", "test", "valid"]:
    nb_lines=sum(1 for _ in open(f"{dest_txt_folder}{split}2id.txt", "r", encoding="utf-8"))
    with open(f"{dest_txt_folder}{split}2id.txt", "r", encoding='utf-8') as r:
        with tqdm(enumerate(r), total=nb_lines) as bar:
            bar.set_description(f"Indexing triples from {split}2id.txt")
            for i, line in bar:
                s, p, o = line.strip().split("\t")
                s, p, o = int(s), int(p), int(o)

                observe(observed_tails_original_kg, s, p, o)
                observe(observed_tails_inv, s, p, o)
                observe(observed_tails_inv, o, p + inverse_predicate_offset, s)

                observe(observed_heads_original_kg, o, p, s)
                observe(observed_heads_inv, o, p, s)
                observe(observed_heads_inv, s, p + inverse_predicate_offset, o)

Indexing triples from valid2id.txt: 100%|██████████████████████████████████████| 5000/5000 [00:00<00:00, 107248.16it/s]


In [19]:
with open(f"{dest_pkl_folder}observed_heads_original_kg.pkl", "wb") as handle:
    dump(observed_heads_original_kg, handle)

with open(f"{dest_pkl_folder}observed_heads_inv.pkl", "wb") as handle:
    dump(observed_heads_inv, handle)

with open(f"{dest_pkl_folder}observed_tails_original_kg.pkl", "wb") as handle:
    dump(observed_tails_original_kg, handle)

with open(f"{dest_pkl_folder}observed_tails_inv.pkl", "wb") as handle:
    dump(observed_tails_inv, handle)

# classid

In [21]:
instances={}

nb_lines=sum(1 for _ in open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTypes.ttl", "r", encoding="utf-8"))
with open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTypes.ttl", "r", encoding='utf-8') as r:
    with tqdm(enumerate(r), total=nb_lines) as bar:
        for i, line in bar:
            if not line.startswith("<"):
                continue
            s, p, o, *_ = line.strip().split("\t")

            if not p == "rdf:type":
                continue

            s = f"http://yago-knowledge.org/resource/{s[1:-1]}"
            o = f"http://yago-knowledge.org/resource/{o[1:-1]}"

            if not s in ents:
                continue

            if not s in instances.keys():
                instances[s]=set([o])
            else:
                instances[s].add(o)

100%|█████████████████████████████████████████████████████████████████| 33854055/33854055 [00:33<00:00, 1011144.93it/s]


In [22]:
{k: list(v)[:5] for k, v in list(instances.items())[:5]}

{'http://yago-knowledge.org/resource/JS_Bordj_Ménaïel': ['http://yago-knowledge.org/resource/wikicat_Sports_clubs_in_Algeria',
  'http://yago-knowledge.org/resource/wikicat_Association_football_clubs_established_in_1932',
  'http://yago-knowledge.org/resource/wordnet_club_108227214',
  'http://yago-knowledge.org/resource/wikicat_Football_clubs_in_Algeria'],
 'http://yago-knowledge.org/resource/Eddie_Kaye_Thomas': ['http://yago-knowledge.org/resource/wikicat_American_male_stage_actors',
  'http://yago-knowledge.org/resource/wikicat_American_people',
  'http://yago-knowledge.org/resource/wikicat_American_television_actors',
  'http://yago-knowledge.org/resource/wikicat_American_actors',
  'http://yago-knowledge.org/resource/wikicat_Actors'],
 'http://yago-knowledge.org/resource/Ahmet_Brković': ['http://yago-knowledge.org/resource/wikicat_Millwall_F.C._players',
  'http://yago-knowledge.org/resource/wikicat_The_Football_League_players',
  'http://yago-knowledge.org/resource/wikicat_HNK_Du

In [23]:
print("Nb of entity in total:", len(ents))
print("Nb of entity with at least a known type:", len(instances.items()))
print("Nb of entity without any known type:", len(ents) - len(instances.items()))

Nb of entity in total: 123182
Nb of entity with at least a known type: 121009
Nb of entity without any known type: 2173


In [24]:
nb_types=sorted(list(set([len(v) for v in instances.values()])))
print("Nb of types per entity", min(nb_types), max(nb_types))

Nb of types per entity 1 101


# Domain / Range information

In [26]:
subproperties={}

nb_lines=sum(1 for _ in open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoSchema.ttl", "r", encoding="utf-8"))
with open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoSchema.ttl", "r", encoding='utf-8') as r:
    with tqdm(enumerate(r), total=nb_lines) as bar:
        for i, line in bar:
            if not line.startswith("<"):
                continue
            s, p, o, *_ = line.strip().split("\t")

            if not p == "rdfs:subPropertyOf":
                continue

            if not s.startswith("<"):
                continue

            if not o.startswith("<"):
                continue

            s = f"http://yago-knowledge.org/resource/{s[1:-1]}"
            o = f"http://yago-knowledge.org/resource/{o[1:-1]}"

            if p == "rdfs:subPropertyOf":
                if not s in subproperties.keys():
                    subproperties[s]=set([])
                subproperties[s].add(o)

100%|████████████████████████████████████████████████████████████████████████████████████████| 985/985 [00:00<?, ?it/s]


In [27]:
def subsumption_closure(c, hierarchy):
    if not c in hierarchy.keys():
        return [c]
    else:
        result = [c]
        for sc in hierarchy[c]:
            result.extend(subsumption_closure(sc, hierarchy))
        return result

In [28]:
subproperty_closure={
    rel: set(subsumption_closure(rel, subproperties))
    for rel in rels
}

In [29]:
properties_of_interest=set(subproperty_closure.keys()).union(*list(subproperty_closure.values()))

In [30]:
domains={}
ranges={}

nb_lines=sum(1 for _ in open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoSchema.ttl", "r", encoding="utf-8"))
with open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoSchema.ttl", "r", encoding='utf-8') as r:
    with tqdm(enumerate(r), total=nb_lines) as bar:
        for i, line in bar:
            if not line.startswith("<"):
                continue
            s, p, o, *_ = line.strip().split("\t")

            if not p in ["rdfs:domain", "rdfs:range", "rdfs:subPropertyOf"]:
                continue

            if not s.startswith("<"):
                continue

            if not o.startswith("<"):
                continue

            s = f"http://yago-knowledge.org/resource/{s[1:-1]}"
            o = f"http://yago-knowledge.org/resource/{o[1:-1]}"

            if not s in properties_of_interest:
                continue

            if p == "rdfs:domain":
                if not s in domains.keys():
                    domains[s]=set([])
                domains[s].add(o)

            if p == "rdfs:range":
                if not s in ranges.keys():
                    ranges[s]=set([])
                ranges[s].add(o)

100%|████████████████████████████████████████████████████████████████████████████████████████| 985/985 [00:00<?, ?it/s]


In [31]:
domains={
    rel: set([]).union(*[
        domains.get(super_rel, set([]))
        for super_rel in subproperty_closure[rel]
    ])
    for rel in rels
}
domains={k: v for k, v in domains.items() if len(v) > 0}

ranges={
    rel: set([]).union(*[
        ranges.get(super_rel, set([]))
        for super_rel in subproperty_closure[rel]
    ])
    for rel in rels
}
ranges={k: v for k, v in ranges.items() if len(v) > 0}

In [32]:
print("Max number of domains per property:", max([len(x) for x in domains.values()]))
print("Max number of ranges per property:", max([len(x) for x in ranges.values()]))

Max number of domains per property: 1
Max number of ranges per property: 1


In [33]:
print("Nb of predicates", len(rels))
print("Nb of domained predicates", len(domains.keys()))
print("Nb of ranged predicates", len(ranges.keys()))

Nb of predicates 37
Nb of domained predicates 37
Nb of ranged predicates 28


In [34]:
{k: v for k, v in list(domains.items())[:5]}

{'http://yago-knowledge.org/resource/livesIn': {'http://yago-knowledge.org/resource/wordnet_person_100007846'},
 'http://yago-knowledge.org/resource/hasMusicalRole': {'http://yago-knowledge.org/resource/wordnet_person_100007846'},
 'http://yago-knowledge.org/resource/participatedIn': {'http://yago-knowledge.org/resource/yagoLegalActorGeo'},
 'http://yago-knowledge.org/resource/wroteMusicFor': {'http://yago-knowledge.org/resource/wordnet_person_100007846'},
 'http://yago-knowledge.org/resource/worksAt': {'http://yago-knowledge.org/resource/wordnet_person_100007846'}}

In [35]:
{k: v for k, v in list(ranges.items())[:5]}

{'http://yago-knowledge.org/resource/livesIn': {'http://yago-knowledge.org/resource/wordnet_location_100027167'},
 'http://yago-knowledge.org/resource/wroteMusicFor': {'http://yago-knowledge.org/resource/wordnet_movie_106613686'},
 'http://yago-knowledge.org/resource/worksAt': {'http://yago-knowledge.org/resource/wordnet_organization_108008335'},
 'http://yago-knowledge.org/resource/playsFor': {'http://yago-knowledge.org/resource/wordnet_organization_108008335'},
 'http://yago-knowledge.org/resource/influences': {'http://yago-knowledge.org/resource/wordnet_person_100007846'}}

In [36]:
domain_inferences={}

for s, po in observed_tails_original_kg.items():
    domain_inferences[id2ent[s]]=set([])
    for p, _ in po.items():
        domain_inferences[id2ent[s]] = domain_inferences[id2ent[s]].union(domains.get(id2rel[p], set([])))

domain_inferences={k: v for k, v in domain_inferences.items() if len(v) > 0}

In [37]:
n=randint(0, len(domain_inferences.keys())-6)
{k: v for k, v in list(domain_inferences.items())[n:n+5]}

{'http://yago-knowledge.org/resource/Branch_County,_Michigan': {'http://yago-knowledge.org/resource/yagoPermanentlyLocatedEntity'},
 'http://yago-knowledge.org/resource/Vijayakumar_(actor)': {'http://yago-knowledge.org/resource/wordnet_actor_109765278',
  'http://yago-knowledge.org/resource/wordnet_person_100007846',
  'http://yago-knowledge.org/resource/yagoLegalActorGeo'},
 'http://yago-knowledge.org/resource/Albert,_Prince_Consort': {'http://yago-knowledge.org/resource/wordnet_person_100007846',
  'http://yago-knowledge.org/resource/yagoLegalActorGeo'},
 'http://yago-knowledge.org/resource/Brian_Green_(footballer)': {'http://yago-knowledge.org/resource/wordnet_person_100007846',
  'http://yago-knowledge.org/resource/yagoLegalActor'},
 'http://yago-knowledge.org/resource/Washington_County_(Maryland)': {'http://yago-knowledge.org/resource/yagoPermanentlyLocatedEntity'}}

In [38]:
range_inferences={}

for o, ps in observed_heads_original_kg.items():
    range_inferences[id2ent[o]]=set([])
    for p, _ in ps.items():
        range_inferences[id2ent[o]] = range_inferences[id2ent[o]].union(ranges.get(id2rel[p], set([])))

range_inferences={k: v for k, v in range_inferences.items() if len(v) > 0}

In [39]:
n=randint(0, len(range_inferences.keys())-6)
{k: v for k, v in list(range_inferences.items())[n:n+5]}

{'http://yago-knowledge.org/resource/University_of_Richmond': {'http://yago-knowledge.org/resource/wordnet_organization_108008335',
  'http://yago-knowledge.org/resource/yagoLegalActor'},
 'http://yago-knowledge.org/resource/Green_Dolphin_Street': {'http://yago-knowledge.org/resource/wordnet_movie_106613686'},
 'http://yago-knowledge.org/resource/FC_Energiya_Chaykovsky': {'http://yago-knowledge.org/resource/wordnet_organization_108008335'},
 'http://yago-knowledge.org/resource/Faten_Hamama': {'http://yago-knowledge.org/resource/wordnet_person_100007846'},
 'http://yago-knowledge.org/resource/Mesa_County,_Colorado': {'http://yago-knowledge.org/resource/yagoGeoEntity'}}

In [40]:
classes_of_interest=set([]).union(*instances.values()).union(*domain_inferences.values()).union(*range_inferences.values())

In [41]:
len(classes_of_interest)

91763

In [42]:
subclasses={}

nb_lines=sum(1 for _ in open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTaxonomy.ttl", "r", encoding="utf-8"))
with open(f"{YAGO3}yago-3.0.2-turtle-simple/yagoTaxonomy.ttl", "r", encoding='utf-8') as r:
    with tqdm(enumerate(r), total=nb_lines) as bar:
        for i, line in bar:
            if not line.startswith("<"):
                continue
            s, p, o, *_ = line.strip().split("\t")

            if not p == "rdfs:subClassOf":
                continue

            if not s.startswith("<"):
                continue

            if not o.startswith("<"):
                continue

            s = f"http://yago-knowledge.org/resource/{s[1:-1]}"
            o = f"http://yago-knowledge.org/resource/{o[1:-1]}"

            if not s in subclasses.keys():
                subclasses[s]=set([])

            subclasses[s].add(o)

100%|████████████████████████████████████████████████████████████████████| 1140933/1140933 [00:01<00:00, 580671.54it/s]


In [43]:
{k: v for k, v in list(subclasses.items())[:5]}

{'http://yago-knowledge.org/resource/wordnet_agape_101028534': {'http://yago-knowledge.org/resource/wordnet_religious_ceremony_101028082'},
 'http://yago-knowledge.org/resource/wikicat_Peace_treaties_of_Great_Britain': {'http://yago-knowledge.org/resource/wordnet_peace_106773976'},
 'http://yago-knowledge.org/resource/wikicat_Former_National_Forests_of_the_United_States': {'http://yago-knowledge.org/resource/wordnet_forest_108438533'},
 'http://yago-knowledge.org/resource/wikicat_Defunct_schools_in_Kingston_upon_Thames': {'http://yago-knowledge.org/resource/wordnet_school_108276720'},
 'http://yago-knowledge.org/resource/wikicat_Islands_of_Tierra_del_Fuego': {'http://yago-knowledge.org/resource/wordnet_island_109316454'}}

In [44]:
subclass_closure={
    cls: set(subsumption_closure(cls, subclasses))
    for cls in classes_of_interest
}

In [45]:
len(subclass_closure.keys())

91763

In [46]:
{k: list(v)[:3] for k, v in list(subclass_closure.items())[:5]}

{'http://yago-knowledge.org/resource/wikicat_Auburn_Tigers_men%27s_basketball_players': ['http://yago-knowledge.org/resource/wordnet_organism_100004475',
  'http://yago-knowledge.org/resource/wikicat_Auburn_Tigers_men%27s_basketball_players',
  'http://yago-knowledge.org/resource/wordnet_athlete_109820263'],
 'http://yago-knowledge.org/resource/wikicat_People_from_Kano': ['http://yago-knowledge.org/resource/wordnet_organism_100004475',
  'http://yago-knowledge.org/resource/wikicat_People_from_Kano',
  'http://yago-knowledge.org/resource/wordnet_physical_entity_100001930'],
 'http://yago-knowledge.org/resource/wikicat_Lone_Star_Soccer_Alliance_players': ['http://yago-knowledge.org/resource/wordnet_organism_100004475',
  'http://yago-knowledge.org/resource/wordnet_physical_entity_100001930',
  'http://yago-knowledge.org/resource/wikicat_Lone_Star_Soccer_Alliance_players'],
 'http://yago-knowledge.org/resource/wikicat_Montreal_Canadiens_coaches': ['http://yago-knowledge.org/resource/wordn

In [47]:
classes_set=set(subclass_closure.keys()).union(*subclass_closure.values())

In [48]:
class2id={cls: i for i, cls in enumerate(classes_set)}
id2class = {i: cls for cls, i in class2id.items()}

with open(f"{dest_pkl_folder}class2id.pkl", "wb") as handle:
    dump(class2id, handle)

In [49]:
r2id2dom2id = {rel2id[rel]: class2id[list(classes)[0]] for rel, classes in domains.items()}
{k: v for k, v in list(r2id2dom2id.items())[:5]}

{0: 84900, 1: 84900, 2: 23427, 3: 84900, 4: 84900}

In [50]:
r2id2range2id = {rel2id[rel]: class2id[list(classes)[0]] for rel, classes in ranges.items()}
{k: v for k, v in list(r2id2range2id.items())[:5]}

{0: 69999, 3: 39154, 4: 36430, 6: 36430, 7: 84900}

In [51]:
with open(f"{dest_pkl_folder}r2id2dom2id.pkl", "wb") as handle:
    dump(r2id2dom2id, handle)

with open(f"{dest_pkl_folder}r2id2range2id.pkl", "wb") as handle:
    dump(r2id2range2id, handle)

In [52]:
instance_no_closure={
    ent: [
        cls
        for cls in instances.get(ent, set([]))\
                    .union(domain_inferences.get(ent, set([])))\
                    .union(range_inferences.get(ent, set([])))
    ]
    for ent in tqdm(ents)
}

100%|██████████████████████████████████████████████████████████████████████| 123182/123182 [00:00<00:00, 272319.91it/s]


In [53]:
{k: list(v)[:5] for k, v in list(instance_no_closure.items())[:5]}

{'http://yago-knowledge.org/resource/Aleksei_Bugayev': ['http://yago-knowledge.org/resource/wikicat_FC_Tom_Tomsk_players',
  'http://yago-knowledge.org/resource/wikicat_FC_Torpedo_Moscow_players',
  'http://yago-knowledge.org/resource/wikicat_Russian_footballers',
  'http://yago-knowledge.org/resource/wikicat_Russian_Premier_League_players',
  'http://yago-knowledge.org/resource/wikicat_FC_Lokomotiv_Moscow_players'],
 'http://yago-knowledge.org/resource/Vincent_Planté': ['http://yago-knowledge.org/resource/wikicat_SM_Caen_players',
  'http://yago-knowledge.org/resource/wikicat_AS_Cannes_players',
  'http://yago-knowledge.org/resource/wikicat_Ligue_1_players',
  'http://yago-knowledge.org/resource/wikicat_French_footballers',
  'http://yago-knowledge.org/resource/wikicat_AS_Saint-Étienne_players'],
 'http://yago-knowledge.org/resource/Anton_Weissenbacher': ['http://yago-knowledge.org/resource/wikicat_FC_Steaua_București_players',
  'http://yago-knowledge.org/resource/wikicat_FC_Baia_Mar

In [54]:
instype={
    ent2id[ent]: [class2id[cls] for cls in classes]
    for ent, classes in instance_no_closure.items()
}

In [55]:
{k: list(v)[:5] for k, v in list(instype.items())[:5]}

{0: [51062, 68289, 56551, 72293, 93050],
 1: [43058, 5948, 28898, 31132, 93720],
 2: [92730, 78497, 67795, 87330, 49138],
 3: [74514, 19309, 90894, 35171, 51664],
 4: [57743, 46617, 19309, 77255, 25263]}

In [56]:
instance_closure_all={
    ent: set([]).union(*[
        subclass_closure[cls]
        for cls in classes
    ])
    for ent, classes in tqdm(instance_no_closure.items())
}

100%|███████████████████████████████████████████████████████████████████████| 123182/123182 [00:01<00:00, 92957.09it/s]


In [57]:
{k: list(v)[:5] for k, v in list(instance_closure_all.items())[:5]}

{'http://yago-knowledge.org/resource/Aleksei_Bugayev': ['http://yago-knowledge.org/resource/wikicat_FC_Tom_Tomsk_players',
  'http://yago-knowledge.org/resource/wikicat_Russian_footballers',
  'http://yago-knowledge.org/resource/wordnet_contestant_109613191',
  'http://yago-knowledge.org/resource/wikicat_Russian_Premier_League_players',
  'http://yago-knowledge.org/resource/wordnet_causal_agent_100007347'],
 'http://yago-knowledge.org/resource/Vincent_Planté': ['http://yago-knowledge.org/resource/wikicat_AS_Saint-Étienne_players',
  'http://yago-knowledge.org/resource/wikicat_AC_Arles-Avignon_players',
  'http://yago-knowledge.org/resource/wordnet_organism_100004475',
  'http://yago-knowledge.org/resource/wordnet_physical_entity_100001930',
  'http://yago-knowledge.org/resource/wordnet_athlete_109820263'],
 'http://yago-knowledge.org/resource/Anton_Weissenbacher': ['http://yago-knowledge.org/resource/wordnet_preserver_110466918',
  'http://yago-knowledge.org/resource/wikicat_Romanian_p

In [58]:
instype_all={
    ent2id[ent]: [class2id[cls] for cls in classes]
    for ent, classes in instance_closure_all.items()
}

In [59]:
{k: list(v)[:5] for k, v in list(instype_all.items())[:5]}

{0: [51062, 56551, 15845, 72293, 59407],
 1: [93720, 30805, 49367, 74269, 30421],
 2: [26320, 16652, 15845, 59407, 53958],
 3: [19309, 87068, 51664, 46617, 53930],
 4: [19309, 91482, 87068, 77255, 5098]}

In [60]:
class2ent={
    cls: [
        ent
        for ent, classes in instance_closure_all.items()
        if cls in classes
    ]
    for cls in tqdm(class2id.keys())
}

100%|████████████████████████████████████████████████████████████████████████████| 93937/93937 [19:44<00:00, 79.31it/s]


In [61]:
{k: list(v)[:5] for k, v in list(class2ent.items())[:5]}

{'http://yago-knowledge.org/resource/wikicat_Auburn_Tigers_men%27s_basketball_players': ['http://yago-knowledge.org/resource/Moochie_Norris',
  'http://yago-knowledge.org/resource/Adam_Harrington_(basketball)',
  'http://yago-knowledge.org/resource/Chris_Morris_(basketball)',
  'http://yago-knowledge.org/resource/Pat_Burke',
  'http://yago-knowledge.org/resource/Mike_Mitchell_(basketball,_born_1956)'],
 'http://yago-knowledge.org/resource/wikicat_People_from_Kano': ['http://yago-knowledge.org/resource/Kelvin_Etuhu',
  'http://yago-knowledge.org/resource/Ismail_Yakubu',
  'http://yago-knowledge.org/resource/Dickson_Etuhu'],
 'http://yago-knowledge.org/resource/wikicat_Lone_Star_Soccer_Alliance_players': ['http://yago-knowledge.org/resource/Thomas_Rongen',
  'http://yago-knowledge.org/resource/Austin_Hudson_(soccer)',
  'http://yago-knowledge.org/resource/Cliff_Brown_(soccer)'],
 'http://yago-knowledge.org/resource/wikicat_Montreal_Canadiens_coaches': ['http://yago-knowledge.org/resource

In [62]:
class2id2ent2id={
    class2id[cls]: [ent2id[ent] for ent in ent_list]
    for cls, ent_list in tqdm(class2ent.items())
}

100%|████████████████████████████████████████████████████████████████████████| 93937/93937 [00:00<00:00, 187473.78it/s]


In [63]:
{k: list(v)[:5] for k, v in list(class2id2ent2id.items())[:5]}

{0: [21480, 32495, 38704, 55757, 55954],
 1: [31710, 88368, 90638],
 2: [56823, 61506, 90093],
 3: [17299, 64109, 100455, 106490],
 4: [66358, 71315, 89738]}

In [64]:
with open(f"{dest_pkl_folder}instype.pkl", "wb") as handle:
    dump(instype, handle)

with open(f"{dest_pkl_folder}instype_all.pkl", "wb") as handle:
    dump(instype_all, handle)

with open(f"{dest_pkl_folder}class2id2ent2id.pkl", "wb") as handle:
    dump(class2id2ent2id, handle)